In [1]:
import xml.etree.cElementTree as ET
import pandas as pd

In [30]:
def read_xml(path, train = True):
    tree = ET.parse(path)
    root = tree.getroot()

    data=[]
    for sentence in root.findall('.//aspectTerms/..'):
        text = sentence.find('text').text
        aspectTerms=sentence.find('aspectTerms')
        for aspectTerm in aspectTerms.findall('aspectTerm'):
            if train:
                term = aspectTerm.get('term')
                polarity = aspectTerm.get('polarity')
                data.append((text, term, polarity))
            else:
                term = aspectTerm.get('term')
                data.append((text, term))
    
    if train:
        df = pd.DataFrame(data,columns=['text', 'term', 'polarity'])
        df = df[df['polarity'].isin(['positive', 'negative', 'neutral'])]
        df['polarity'] = df['polarity'].map(
            {'positive': 1, 'neutral': 0, 'negative': -1})
    else:
        df = pd.DataFrame(data,columns=['text', 'term'])

    return df

In [31]:
laptop_train = read_xml('Laptop_Train_v2.xml')

In [32]:
rest_train = read_xml('Restaurants_Train_v2.xml')

In [17]:
def read_txt(path):
    temp = pd.read_table(path, header = None)
    text = []
    term = []
    polarity = []
    for i in range(len(temp)):
        if i % 3 == 0:
            text.append(temp.iloc[i,0].strip())
        elif i % 3 == 1:
            term.append(temp.iloc[i,0].strip())
        else:
            polarity.append(temp.iloc[i,0])
    
    df = pd.DataFrame()
    df['text'] = text
    df['term'] = term
    df['polarity'] = polarity
    def replace_str(x, y):
        x = x.replace('$T$',y)
        return x
    df['text'] = df.apply(lambda x: replace_str(x['text'],x['term']),axis = 1)
    return df

In [19]:
laptop_test = read_txt('Laptops_Test_Gold.txt')
rest_test = read_txt('Restaurants_Test_Gold.txt')

In [28]:
laptop_test.to_csv('laptop_test.csv', index = 0)

In [29]:
rest_test.to_csv('rest_test.csv', index = 0)

In [34]:
laptop_train.to_csv('laptop_train.csv', index = 0)

In [35]:
rest_train.to_csv('rest_train.csv', index = 0)